In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

# --- 1. 데이터 임포트 ---
from services.tables.HR_Core.basic_info_table import emp_df
from services.tables.HR_Core.job_info_table import job_info_df
from services.tables.HR_Core.department_info_table import department_info_df
from services.tables.HR_Core.job_table import job_df, job_df_indexed, parent_map_job, job_l1_order
from services.tables.HR_Core.department_table import division_order, dept_level_map, parent_map_dept, dept_name_map
from services.helpers.utils import get_level1_ancestor, find_division_name_for_dept

def create_figure_and_df():
    """
    제안 8: 직무별 인력 유지 현황 분석 그래프 및 피벗 테이블을 생성합니다.
    """
    # --- 2. 데이터 준비 및 가공 ---
    emp_base_df = emp_df[['EMP_ID', 'CURRENT_EMP_YN', 'DURATION']].copy()
    emp_base_df['TENURE_YEARS'] = emp_base_df['DURATION'] / 365.25

    last_job = job_info_df.sort_values('JOB_APP_START_DATE').groupby('EMP_ID').last()
    last_dept = department_info_df.sort_values('DEP_APP_START_DATE').groupby('EMP_ID').last()

    job_l1_map = job_df[job_df['JOB_LEVEL'] == 1].set_index('JOB_ID')['JOB_NAME'].to_dict()
    last_job['JOB_CATEGORY'] = last_job['JOB_ID'].apply(lambda x: job_l1_map.get(get_level1_ancestor(x, job_df_indexed, parent_map_job)))
    last_dept['DIVISION_NAME'] = last_dept['DEP_ID'].apply(lambda x: find_division_name_for_dept(x, dept_level_map, parent_map_dept, dept_name_map))

    analysis_df = pd.merge(emp_base_df, last_job[['JOB_CATEGORY']], on='EMP_ID', how='left')
    analysis_df = pd.merge(analysis_df, last_dept[['DIVISION_NAME']], on='EMP_ID', how='left')
    analysis_df = analysis_df.dropna(subset=['JOB_CATEGORY', 'DIVISION_NAME'])
    analysis_df['STATUS'] = np.where(analysis_df['CURRENT_EMP_YN'] == 'Y', '재직자', '퇴사자')

    summary_df = analysis_df.groupby(['DIVISION_NAME', 'JOB_CATEGORY', 'STATUS'], observed=False).agg(
        AVG_TENURE=('TENURE_YEARS', 'mean'),
        HEADCOUNT=('EMP_ID', 'nunique')
    ).unstack(level='STATUS').fillna(0)
    summary_df.columns = [f'{val}_{stat}' for val, stat in summary_df.columns]
    summary_df = summary_df.reset_index()

    overall_summary = analysis_df.groupby(['JOB_CATEGORY', 'STATUS'], observed=False).agg(
        AVG_TENURE=('TENURE_YEARS', 'mean'),
        HEADCOUNT=('EMP_ID', 'nunique')
    ).unstack(level='STATUS').fillna(0)
    overall_summary.columns = [f'{val}_{stat}' for val, stat in overall_summary.columns]
    overall_summary = overall_summary.reset_index()
    
    x_max = pd.concat([summary_df['AVG_TENURE_재직자'], summary_df['AVG_TENURE_퇴사자']]).max()
    fixed_x_range = [0, x_max * 1.15]

    # --- 3. Plotly 인터랙티브 그래프 생성 ---
    fig = go.Figure()
    division_list = ['전체'] + division_order
    for i, div_name in enumerate(division_list):
        is_visible = (i == 0)
        if div_name == '전체':
            df_grouped = overall_summary
        else:
            df_grouped = summary_df[summary_df['DIVISION_NAME'] == div_name]
        fig.add_trace(go.Bar(
            y=df_grouped['JOB_CATEGORY'], x=df_grouped['AVG_TENURE_재직자'], name='재직자', orientation='h', visible=is_visible,
            customdata=df_grouped['HEADCOUNT_재직자'], text=df_grouped['AVG_TENURE_재직자'].round(2),
            textposition='outside', hovertemplate='평균 재직기간: %{x:.2f}년<br>인원: %{customdata}명<extra></extra>'
        ))
        fig.add_trace(go.Bar(
            y=df_grouped['JOB_CATEGORY'], x=df_grouped['AVG_TENURE_퇴사자'], name='퇴사자', orientation='h', visible=is_visible,
            customdata=df_grouped['HEADCOUNT_퇴사자'], text=df_grouped['AVG_TENURE_퇴사자'].round(2),
            textposition='outside', hovertemplate='평균 재직기간: %{x:.2f}년<br>인원: %{customdata}명<extra></extra>'
        ))

    # --- 4. 드롭다운 메뉴 생성 및 레이아웃 업데이트 ---
    buttons = []
    for i, div_name in enumerate(division_list):
        visibility_mask = [False] * (len(division_list) * 2)
        visibility_mask[i*2], visibility_mask[i*2 + 1] = True, True
        buttons.append(dict(label=div_name, method='update', args=[{'visible': visibility_mask}]))
    
    fig.update_layout(
        updatemenus=[dict(active=0, buttons=buttons, direction="down", pad={"r": 10, "t": 10}, showactive=True, x=0.01, xanchor="left", y=1.1, yanchor="top")],
        title_text='직무별 평균 재직기간 비교 (재직자 vs 퇴사자)',
        xaxis_title='평균 재직 기간 (년)',
        font_size=14, height=700,
        barmode='group',
        legend_title_text='상태',
        annotations=[dict(text="조직 선택:", showarrow=False, x=0, y=1.08, yref="paper", align="left")],
        xaxis_range=fixed_x_range,
        # --- 수정된 부분: global_job_order 대신 job_l1_order 사용 ---
        yaxis=dict(
            title='마지막 직무 대분류',
            categoryorder='array',
            categoryarray=job_l1_order[::-1]
        )
        # --- 수정 완료 ---
    )
    
    # --- aggregate_df 생성 ---
    analysis_df['STATUS'] = pd.Categorical(analysis_df['STATUS'], categories=['퇴사자', '재직자'], ordered=True)
    aggregate_df = analysis_df.pivot_table(
        index=['JOB_CATEGORY', 'STATUS'], columns='DIVISION_NAME',
        values='TENURE_YEARS', aggfunc='mean', observed=False
    )
    aggregate_df['전체 평균'] = analysis_df.groupby(['JOB_CATEGORY', 'STATUS'], observed=False)['TENURE_YEARS'].mean()
    cols = ['전체 평균'] + [col for col in division_order if col in aggregate_df.columns]
    aggregate_df = aggregate_df[cols].round(2)
        
    return fig, aggregate_df

# --- 실행 코드 ---
pio.renderers.default = 'vscode'
fig, aggregate_df = create_figure_and_df()
fig.show()

print("\n--- Aggregate DataFrame ---")
aggregate_df


--- Aggregate DataFrame ---


DIVISION_NAME                    전체 평균  Planning Division  Sales Division  \
JOB_CATEGORY             STATUS                                             
IT                       퇴사자      4.68               4.66            3.96   
                         재직자      5.41               5.09            5.71   
Management Support       퇴사자      5.02               5.12            3.59   
                         재직자      5.45               5.40            5.27   
Planning                 퇴사자      5.06               5.10            4.80   
                         재직자      5.64               6.40            5.36   
Production & Engineering 퇴사자      4.61               6.85            5.55   
                         재직자      5.29               5.38            5.44   
Sales & Marketing        퇴사자      4.67               5.00            4.76   
                         재직자      5.25               4.32            5.36   

DIVISION_NAME                    Development Division  Operating Division  
JOB_CATEGORY             STATUS                                            
IT                       퇴사자                     5.07                4.55  
                         재직자                     5.88                4.79  
Management Support       퇴사자                     5.86                 NaN  
                         재직자                     6.22                6.31  
Planning                 퇴사자                     4.85                5.75  
                         재직자                     5.21                3.39  
Production & Engineering 퇴사자                     4.49                4.38  
                         재직자                     4.97                5.51  
Sales & Marketing        퇴사자                     2.82                4.60  
                         재직자                     4.60                5.30